In [ ]:
from flask import Flask, jsonify
import pymysql
import torch
import os
import requests  # ✅ 추가
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

app = Flask(__name__)

# ✅ 디바이스 설정 (GPU가 있으면 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Fine-tuned CLIP 모델 로드
model = CLIPModel.from_pretrained("./clip_finetuned_model").to(device)
processor = CLIPProcessor.from_pretrained("./clip_finetuned_model")
model.eval()

# ✅ 분류 대상 태그 클래스 (수정 가능)
class_names = ["food", "people", "landscape", "accommodation"]

# ✅ 이미지 분류 함수 (image_path → image_url로 변경)
def predict_tag(image_url):
    image = Image.open(requests.get(image_url, stream=True).raw).convert("RGB")
    inputs = processor(text=class_names, images=image, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = outputs.logits_per_image.softmax(dim=1)
        pred_index = torch.argmax(probs).item()
        return class_names[pred_index]

# ✅ MySQL DB 접속 설정 (포트 3307)
DB_CONFIG = {
    'host': 'project-db-cgi.smhrd.com',
    'port': 3307,
    'user': 'cgi_24K_AI4_p3_2',
    'password': 'smhrd2',  # ← 본인 비밀번호로 변경
    'db': 'cgi_24K_AI4_p3_2',         # ← 본인 DB 이름으로 변경
    'charset': 'utf8mb4'
}

# ✅ VSCode 서버 쪽 uploads 폴더 절대 경로 (이제 사용 안함)
UPLOADS_DIR = "C:/Users/smhrd/Desktop/pokachip/server/uploads"

# ✅ Flask API 엔드포인트: /classify
@app.route('/classify', methods=['POST'])
def classify_images():
    conn = pymysql.connect(**DB_CONFIG)
    cursor = conn.cursor(pymysql.cursors.DictCursor)

    # tags가 비어있는 항목들만 선택
    cursor.execute("SELECT photo_idx, file_name FROM photo_info WHERE tags IS NULL OR tags = ''")
    photos = cursor.fetchall()

    classified_count = 0

    for photo in photos:
        photo_idx = photo['photo_idx']
        # 슬래시 정규화 + 파일명만 추출
        filename = os.path.basename(photo['file_name'].replace('\\', '/'))

        # ✅ 이미지 URL 경로로 변경 (Node.js 서버 주소로 바꿔야 함)
        image_url = f"https://your-node-app.onrender.com/uploads/{filename}"

        try:
            tag = predict_tag(image_url)
            cursor.execute("UPDATE photo_info SET tags = %s WHERE photo_idx = %s", (tag, photo_idx))
            conn.commit()
            print(f"✅ 분류 완료: {filename} → {tag}")
            classified_count += 1
        except Exception as e:
            print(f"⚠️ 예측 실패 ({filename}): {e}")

    cursor.close()
    conn.close()

    return jsonify({
        "status": "success",
        "classified": classified_count
    })

# ✅ 서버 실행
if __name__ == '__main__':
    app.run(port=6006)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:6006
Press CTRL+C to quit


❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748681995036-1748218298711-1747985122592-KIA.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748681995041-1748313651762-1748248087723-bam.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748995851305-1747985122592-KIA.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748995851320-1747985357221-test2.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748995851330-1748227248817-BALL.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748999477645-KakaoTalk_20250601_082515601.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748999477665-KakaoTalk_20250601_082515601_01.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748999477680-KakaoTalk_20250601_082515601_02.jpg
✅ 분류 완료: 1749395903400-2.jpg → people


127.0.0.1 - - [09/Jun/2025 00:18:38] "POST /classify HTTP/1.1" 200 -


✅ 분류 완료: 1749395903427-3.jpg → landscape
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748681995036-1748218298711-1747985122592-KIA.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748681995041-1748313651762-1748248087723-bam.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748995851305-1747985122592-KIA.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748995851320-1747985357221-test2.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748995851330-1748227248817-BALL.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748999477645-KakaoTalk_20250601_082515601.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748999477665-KakaoTalk_20250601_082515601_01.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1748999477680-KakaoTalk_20250601_082515601_02.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server/uploads\1749428791415-KakaoTalk_20250603_123523901_03.jpg
❌ 파일 없음: C:/Users/smhrd/Desktop/pokachip/server

127.0.0.1 - - [09/Jun/2025 09:30:13] "POST /classify HTTP/1.1" 200 -


✅ 분류 완료: 1749428997757-1748503604085-KakaoTalk_20250529_162616991_01.jpg → landscape


In [1]:
pip install transformers

   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------------------------------------  10.2/10.5 MB 53.3 MB/s eta 0:00:01
   ---------------------------------------- 10.5/10.5 MB 36.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 46.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install pymysql

In [5]:
# 셀 안에서 실행
!jupyter nbconvert --to script flask_server.ipynb

Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\Scripts\jupyter-nbconvert-script.py", line 6, in <module>
    from nbconvert.nbconvertapp import main
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\nbconvert\nbconvertapp.py", line 193, in <module>
    class NbConvertApp(JupyterApp):
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\nbconvert\nbconvertapp.py", line 252, in NbConvertApp
    Options include {get_export_names()}.
                     ^^^^^^^^^^^^^^^^^^
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\nbconvert\exporters\base.py", line 145, in get_export_names
    e = get_exporter(exporter_name)(config=config)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\nbconvert\exporters\base.py", line 106, in get_exporter
    exporter = items[0].load()
               ^^^^^^^^^^^^^^^
  File "C:\Users\smhrd\anaconda3\Lib\importlib\metadata\__init__.py", line 205, in load
    module = import_module(match.group('module'))
 

In [3]:
pip install notebook

Note: you may need to restart the kernel to use updated packages.
